###**Videos to frames**

In [ ]:

#2chamber video


import cv2
import os


def convert_video_to_frames(input_video_path, output_folder, num_frames=150):
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame interval to get exactly num_frames frames
    frame_interval = max(1, total_frames // num_frames)

    # Initialize a counter for naming the frames and a counter for saved frames
    frame_counter = 0
    saved_frames = 0

    # Loop through the video frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Only save frames at the calculated interval
        if frame_counter % frame_interval == 0:
            # Save the frame as an image
            frame_path = os.path.join(output_folder, f'frame_{saved_frames:04d}.jpg')
            cv2.imwrite(frame_path, frame)

            saved_frames += 1

            # Break if we have saved num_frames frames
            if saved_frames == num_frames:
                break

        # Increment the frame counter
        frame_counter += 1

    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Usage
input_video_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/input/2ch_1.avi'
output_folder = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames/2'
num_frames = 150
convert_video_to_frames(input_video_path, output_folder, num_frames)


#4chamber video


import cv2
import os

def convert_video_to_frames(input_video_path, output_folder, num_frames=150):
    # Open the video file
    cap = cv2.VideoCapture(input_video_path)

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the frame interval to get exactly num_frames frames
    frame_interval = max(1, total_frames // num_frames)

    # Initialize a counter for naming the frames and a counter for saved frames
    frame_counter = 0
    saved_frames = 0

    # Loop through the video frames
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Only save frames at the calculated interval
        if frame_counter % frame_interval == 0:
            # Save the frame as an image
            frame_path = os.path.join(output_folder, f'frame_{saved_frames:04d}.jpg')
            cv2.imwrite(frame_path, frame)

            saved_frames += 1

            # Break if we have saved num_frames frames
            if saved_frames == num_frames:
                break

        # Increment the frame counter
        frame_counter += 1

    # Release the video capture object and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Usage
input_video_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/input/4ch_1.avi'
output_folder = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames/4'
num_frames = 150
convert_video_to_frames(input_video_path, output_folder, num_frames)


###**Image Processing**

In [ ]:
import os
import cv2
from google.colab import drive

def apply_bilateral_filter(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the input folder
    input_files = os.listdir(input_folder)

    for filename in input_files:
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            # Read the frame
            frame_path = os.path.join(input_folder, filename)
            img = cv2.imread(frame_path)

            # Apply bilateral filter
            bilateral = cv2.bilateralFilter(img, d=15, sigmaColor=75, sigmaSpace=75)

            # Save the filtered frame to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, bilateral)




# Define input and output folder paths in Google Drive
input_folder_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames/2'
output_folder_path= '/content/drive/MyDrive/all work-BACKUP/automation folder/image processed images/2ch'

apply_bilateral_filter(input_folder_path, output_folder_path)


import os
import cv2
from google.colab import drive

def apply_bilateral_filter(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the input folder
    input_files = os.listdir(input_folder)

    for filename in input_files:
        if filename.endswith(('.jpg', '.png', '.jpeg')):
            # Read the frame
            frame_path = os.path.join(input_folder, filename)
            img = cv2.imread(frame_path)

            # Apply bilateral filter
            bilateral = cv2.bilateralFilter(img, d=15, sigmaColor=75, sigmaSpace=75)

            # Save the filtered frame to the output folder
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, bilateral)




# Define input and output folder paths in Google Drive
input_folder_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames/4'
output_folder_path= '/content/drive/MyDrive/all work-BACKUP/automation folder/image processed images/4ch'

apply_bilateral_filter(input_folder_path, output_folder_path)

###**Automatic Segmentation**

In [ ]:
import os
import sys
import datetime
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as keras
import tensorflow.keras.backend as K
from tensorflow import argmax
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split

# Utils
import h5py

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import load_model

#2chamber

# Load the pre-trained UNet model
model_unet = tf.keras.models.load_model("/content/drive/MyDrive/all work-BACKUP/automation folder/model/Unet_model_2ch.hdf5", compile=False)

# Function to process images from the input folder and save segmented masks into the output folder
def process_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check if the file is an image file
            # Load the image
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Resize the image to dimensions divisible by 32 and match the expected input shape of the model
            resized_img = cv2.resize(img, (384, 384))

            # Normalize the image
            normalized_img = resized_img.astype(np.float32) / 255.0

            # Expand dimensions to add channel dimension
            input_img = np.expand_dims(normalized_img, axis=-1)

            # Predict mask using the UNet model
            prediction = model_unet.predict(np.expand_dims(input_img, axis=0))
            predicted_mask = tf.argmax(prediction, axis=-1).numpy().squeeze()

            # Save the predicted mask as an image
            mask_filename = f"{os.path.splitext(filename)[0]}_mask.png"
            mask_path = os.path.join(output_folder, mask_filename)
            plt.imsave(mask_path, predicted_mask, cmap='gray')


# Define the input and output folders
input_folder = "/content/drive/MyDrive/all work-BACKUP/automation folder/image processed images/2ch"
output_folder = "/content/drive/MyDrive/all work-BACKUP/automation folder/segmented masks/2ch"

# Process images from the input folder and save segmented masks into the output folder
process_images(input_folder, output_folder)

#4chamber

# Load the pre-trained UNet model
model_unet = tf.keras.models.load_model("/content/drive/MyDrive/all work-BACKUP/automation folder/model/Unet_model_4ch.hdf5", compile=False)

# Function to process images from the input folder and save segmented masks into the output folder
def process_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):  # Check if the file is an image file
            # Load the image
            img_path = os.path.join(input_folder, filename)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            # Resize the image to dimensions divisible by 32 and match the expected input shape of the model
            resized_img = cv2.resize(img, (384, 384))

            # Normalize the image
            normalized_img = resized_img.astype(np.float32) / 255.0

            # Expand dimensions to add channel dimension
            input_img = np.expand_dims(normalized_img, axis=-1)

            # Predict mask using the UNet model
            prediction = model_unet.predict(np.expand_dims(input_img, axis=0))
            predicted_mask = tf.argmax(prediction, axis=-1).numpy().squeeze()

            # Save the predicted mask as an image
            mask_filename = f"{os.path.splitext(filename)[0]}_mask.png"
            mask_path = os.path.join(output_folder, mask_filename)
            plt.imsave(mask_path, predicted_mask, cmap='gray')


# Define the input and output folders
input_folder = "/content/drive/MyDrive/all work-BACKUP/automation folder/image processed images/4ch"
output_folder = "/content/drive/MyDrive/all work-BACKUP/automation folder/segmented masks/4ch"

# Process images from the input folder and save segmented masks into the output folder
process_images(input_folder, output_folder)


1/1 [==============================] - 4s 4s/step


###**Frame Selection**

In [ ]:
import os
import cv2
import numpy as np
import shutil
import matplotlib.pyplot as plt

# Function to select end-systolic and end-diastolic frames using frame differencing
def select_frames(input_folder_2ch, input_folder_4ch, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process 2ch frames
    process_frames(input_folder_2ch, os.path.join(output_folder, "2ch"))

    # Process 4ch frames
    process_frames(input_folder_4ch, os.path.join(output_folder, "4ch"))

# Function to process frames in a given input folder
def process_frames(input_folder, output_subfolder):
    # Create the output subfolder
    os.makedirs(output_subfolder, exist_ok=True)

    # Get the list of mask files sorted by modification time
    mask_files = sorted(os.listdir(input_folder), key=lambda f: os.path.getmtime(os.path.join(input_folder, f)))
    num_frames = len(mask_files)

    # Calculate frame differences
    frame_diffs = []
    for i in range(1, num_frames):
        prev_frame = cv2.imread(os.path.join(input_folder, mask_files[i-1]), cv2.IMREAD_GRAYSCALE)
        curr_frame = cv2.imread(os.path.join(input_folder, mask_files[i]), cv2.IMREAD_GRAYSCALE)
        diff = np.mean(np.abs(prev_frame - curr_frame))
        frame_diffs.append(diff)

    # Find indices of frames with maximum and minimum differences
    end_systolic_frame_idx = np.argmax(frame_diffs)
    end_diastolic_frame_idx = np.argmin(frame_diffs)

    # Copy end-systolic and end-diastolic frames to the output folder
    shutil.copy2(os.path.join(input_folder, mask_files[end_systolic_frame_idx]), os.path.join(output_subfolder, f'end_systolic_frame.png'))
    shutil.copy2(os.path.join(input_folder, mask_files[end_diastolic_frame_idx]), os.path.join(output_subfolder, f'end_diastolic_frame.png'))


# Define input and output folders
input_folder_2ch = "/content/drive/MyDrive/all work-BACKUP/automation folder/segmented masks/2ch"
input_folder_4ch = "/content/drive/MyDrive/all work-BACKUP/automation folder/segmented masks/4ch"
output_folder = "/content/drive/MyDrive/all work-BACKUP/automation folder/frames images selected "

# Select end-systolic and end-diastolic frames and save them into output folders
select_frames(input_folder_2ch, input_folder_4ch, output_folder)



###**Ejection Fraction**

In [ ]:

import cv2
import numpy as np

#
# Function to preprocess the image
def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred

# Function to segment ventricles
def segment_ventricles(image):
    # Apply thresholding to segment ventricles
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Find contours of segmented regions
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Sort contours by area and find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    # Create a mask for the largest contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
    return mask

# Function to calculate volume from segmented mask
def calculate_volume(mask, pixel_spacing):
    # Calculate volume in cubic millimeters (mm^3)
    volume = np.sum(mask) * pixel_spacing[0] * pixel_spacing[1] * pixel_spacing[2]
    return volume

# Function to process and calculate volume for each image
def process_and_calculate_volume(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    # Segment ventricles
    ventricle_mask = segment_ventricles(preprocessed_image)
    # Calculate pixel spacing (assuming isotropic voxels)
    pixel_spacing = (1.0, 1.0, 1.0)  # Example pixel spacing in millimeters (x, y, z)
    # Calculate volume
    volume = calculate_volume(ventricle_mask, pixel_spacing)
    return volume

# Paths to the images
four_chamber_systole_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames images selected /4ch/end_systolic_frame.png'
two_chamber_systole_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames images selected /2ch/end_systolic_frame.png'

# Process and calculate volumes for each image
four_chamber_systole_volume = process_and_calculate_volume(four_chamber_systole_path)
two_chamber_systole_volume = process_and_calculate_volume(two_chamber_systole_path)

# Print the calculated volumes
ESV=(two_chamber_systole_volume+four_chamber_systole_volume)//2





import cv2
import numpy as np



# Function to preprocess the image
def preprocess_image(image):
    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    return blurred

# Function to segment ventricles
def segment_ventricles(image):
    # Apply thresholding to segment ventricles
    _, thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Find contours of segmented regions
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # Sort contours by area and find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    # Create a mask for the largest contour
    mask = np.zeros_like(image)
    cv2.drawContours(mask, [largest_contour], -1, 255, thickness=cv2.FILLED)
    return mask

# Function to calculate volume from segmented mask
def calculate_volume(mask, pixel_spacing):
    # Calculate volume in cubic millimeters (mm^3)
    volume = np.sum(mask) * pixel_spacing[0] * pixel_spacing[1] * pixel_spacing[2]
    return volume

# Function to process and calculate volume for each image
def process_and_calculate_volume(image_path):
    # Read the image
    image = cv2.imread(image_path)
    # Preprocess the image
    preprocessed_image = preprocess_image(image)
    # Segment ventricles
    ventricle_mask = segment_ventricles(preprocessed_image)
    # Calculate pixel spacing (assuming isotropic voxels)
    pixel_spacing = (1.2, 1.2, 1.2)  # Example pixel spacing in millimeters (x, y, z)
    # Calculate volume
    volume = calculate_volume(ventricle_mask, pixel_spacing)
    return volume

# Paths to the images
four_chamber_diastole_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames images selected /4ch/end_diastolic_frame.png'
two_chamber_diastole_path = '/content/drive/MyDrive/all work-BACKUP/automation folder/frames images selected /2ch/end_diastolic_frame.png'

# Process and calculate volumes for each image
four_chamber_diastole_volume = process_and_calculate_volume(four_chamber_diastole_path)
two_chamber_diastole_volume = process_and_calculate_volume(two_chamber_diastole_path)

# Print the calculated volumes
EDV=(two_chamber_diastole_volume+four_chamber_diastole_volume)//2

EF=((EDV-ESV)/EDV)*100
EF=round(EF,2)


gender = input("Enter gender (male/female): ").lower()
age=int(input("Enter age = "))
male_ef_ranges = {
    "Normal": (52, 72),
    "Mildly Reduced": (41, 51),
    "Moderately Reduced": (30, 40),
    "Severely Reduced": (0, 30)
}

female_ef_ranges = {
    "Normal": (54, 75),
    "Mildly Reduced": (41, 53),
    "Moderately Reduced": (30, 40),
    "Severely Reduced": (0, 30)
}

if gender == "male":
    if EF >= male_ef_ranges["Normal"][0] and EF <= male_ef_ranges["Normal"][1]:
        print("Male EF",EF,"%"," gives heart condition is normal. ")
    elif EF >= male_ef_ranges["Mildly Reduced"][0] and EF <= male_ef_ranges["Mildly Reduced"][1]:
        print("Male EF",EF,"%"," gives heart condition is Mildly abnormal. ")
    elif EF >= male_ef_ranges["Moderately Reduced"][0] and EF <= male_ef_ranges["Moderately Reduced"][1]:
        print("Male EF",EF,"%"," gives heart condition is Moderately abnormal. ")
    else:
        print("Male EF",EF,"%"," gives heart condition is Severe. ")

elif gender == "female":
    if EF >= female_ef_ranges["Normal"][0] and EF <= female_ef_ranges["Normal"][1]:
        print("Female EF",EF,"%","gives heart condition is Normal")
    elif EF >= female_ef_ranges["Mildly Reduced"][0] and EF <= female_ef_ranges["Mildly Reduced"][1]:
        print("Female EF",EF,"%","gives heart condition is Mildy Abnormal")
    elif EF >= female_ef_ranges["Moderately Reduced"][0] and EF <= female_ef_ranges["Moderately Reduced"][1]:
        print("Female EF",EF,"%","gives heart condition is Moderately Abnormal")
    else:
        print("Female EF",EF,"%","gives heart condition is Severe")
else:
    print("Invalid input. Please enter 'male' or 'female' as the gender.")


Enter gender (male/female): male
Enter age = 45
Male EF 53.46 %  gives heart condition is normal. 
